In [1]:
!pip install transformers[sentencepiece]==4.28.0 datasets sacrebleu evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 63.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.29.2
    Uninstalling transformers-4.29.2:
      Successfully uninstalled transformers-4.29.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.4.1 requires cupy-cuda11x<12.0.0a0,>=9.5.0, which is not installed.
cuml 23.4.1 requires cupy-cuda11x<12.0.0a0,>=9.5.0, which is not installed.
dask-cudf 23

In [2]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
import wandb
user_secrets = UserSecretsClient()
secret_value = user_secrets.get_secret("HF_KEY")
login(secret_value)
secret_value1 = user_secrets.get_secret("WANDB_KEY")
wandb.login(key=secret_value1)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid.
Your token has been saved to /root/.cache/huggingface/token
Login successful


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
import transformers
print(transformers.__version__)

4.28.0


In [4]:
#from datasets import load_dataset
#wmt16_train = load_dataset("ethansimrm/wmt16_biomed", use_auth_token=True) #We won't need to use this for now
#wmt16_test = load_dataset("ethansimrm/wmt16_biomed_test", use_auth_token=True)
#wmt16_gold = load_dataset("ethansimrm/wmt16_biomed_gold", use_auth_token=True)

Let's aim low for now. Given a (small) pre-trained large language model - Marian or T5-Small, etc, we wish to evaluate baseline performance on the WMT16 Biomedical Test Set. 
- We have converted the test set and the gold-standard answers to CSV files and uploaded both to the HuggingFace Hub.
- Intuitively, we must extract source sentences from the test set and target sentences from the gold set, but only where there is a direct correspondence between them.
- We know for sure that "passage/0" (the scientific paper titles) - are one sentence long in both languages, so there is a direct correspondence - we cannot guarantee the same for the rest of the sentences.



Additional Information:
- We could simply truncate the gold-standard answers to the length of the source sentences, but we only require a quick-and-dirty evaluation today.
- Furthermore, the test set contains French abstracts which contain information which cannot be inferred from the English source; we cannot assume a 1-1 faithful translation.

In [5]:
#The relevant columns for the abstract titles are:
#source_sentences = wmt16_test["test"]["passage/0/sentence/0/text"]
#target_sentences = wmt16_gold["train"]["passage/0/sentence/0/text"]

In [6]:
from datasets import load_metric
metric = load_metric("sacrebleu")

In [7]:
#This used a pretrained model from an earlier tutorial - it's quite bad. We need to train our base models
#on at least some out of domain EN-FR data, and upload them to the Hub before evaluating on the test set.
#from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, T5Model, pipeline
#tokenizer = AutoTokenizer.from_pretrained("t5-small")
#model = AutoModelForSeq2SeqLM.from_pretrained("ethansimrm/test_t5_small_example_kaggle3") 
#translator = pipeline("translation", model=model, tokenizer=tokenizer, use_auth_token = True)
#for s in source_sentences:
    #source = "translate English to French: " + s
    #if(source[len(source) - 1] == '.'):
        #source = source[:-1]
    #print(source)
    #result = translator(source)
    #print(result)

The usual corpora used in research are massive. Let's get a (much) smaller one out - the EMEA corpus, which also happens to be in-domain, and is a component of many concatenated training corpora!
If time permits, we can try again with the UN Parallel Corpus (9GB unpacked) or the WMT14 EN-FR parallel corpus for more out-of-domain training, which SPECTRANS showed to be indispensable.

In [8]:
from datasets import load_dataset
training_data = load_dataset("emea", lang1="en", lang2="fr")

Generating train split: 0 examples [00:00, ? examples/s]

Dataset emea downloaded and prepared to /root/.cache/huggingface/datasets/emea/en-fr-lang1=en,lang2=fr/0.0.0/346201b3d2bac55605dad3ed8853320c0b12a4ba0177d1e3c1ae12b8bad6d9a9. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
training_data = training_data['train'].train_test_split(test_size=0.2)

In [10]:
training_data['train'][0]

{'id': '464389',
 'translation': {'en': 'These measures will help to protect the environment.',
  'fr': 'Ces mesures permettront de protéger l’ environnement.'}}

In [11]:
from transformers import AutoTokenizer
checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [12]:
source_lang = "en"
target_lang = "fr"
prefix = "translate English to French: "
def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]] 
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [13]:
tokenized_train = training_data.map(preprocess_function, batched=True)

  0%|          | 0/875 [00:00<?, ?ba/s]

  0%|          | 0/219 [00:00<?, ?ba/s]

In [14]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [15]:
import numpy as np

def postprocess_text(preds, labels): 
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True) #Convert back into words

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id) #Ignore padded labels added by the data collator to the test set
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True) 

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels) #Remove leading and trailing spaces

    result = metric.compute(predictions=decoded_preds, references=decoded_labels) #BLEU score for provided input and references
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens) #Compute mean prediction length
    result = {k: round(v, 4) for k, v in result.items()} #Round score to 4dp
    return result

In [16]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [17]:
import torch

training_args = Seq2SeqTrainingArguments( #Collects hyperparameters
    output_dir="t5_small_prelim_emea_enfr",
    evaluation_strategy="epoch", #Evaluates at the end of each epoch
    learning_rate=2e-5, #Initial learning rate for AdamW
    per_device_train_batch_size=16, #Minibatch learning
    per_device_eval_batch_size=16, #Batch size for evaluation
    weight_decay=0.01, #Weight decay for loss computation; Loss = Loss + WD * sum (weights squared)
    save_total_limit=3, #Number of checkpoints to save
    num_train_epochs=2,
    predict_with_generate=True, #Use with ROUGE/BLEU and other translation metrics (see below)
    fp16=True, #Remove fp16 = True if not using CUDA
    push_to_hub=True,
)

trainer = Seq2SeqTrainer( #Saves us from writing our own training loops
    model=model,
    args=training_args,
    train_dataset=tokenized_train["train"],
    eval_dataset=tokenized_train["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Cloning https://huggingface.co/ethansimrm/t5_small_prelim_emea_enfr into local empty directory.


Download file pytorch_model.bin:   0%|          | 8.00k/231M [00:00<?, ?B/s]

Download file runs/May26_12-00-38_3fa6313d89f7/1685102444.3299024/events.out.tfevents.1685102444.3fa6313d89f7.…

Download file spiece.model:   3%|3         | 24.0k/773k [00:00<?, ?B/s]

Download file runs/May26_12-03-48_3fa6313d89f7/1685102630.7232652/events.out.tfevents.1685102630.3fa6313d89f7.…

...

Clean file runs/May26_12-00-38_3fa6313d89f7/events.out.tfevents.1685102444.3fa6313d89f7.28.2:  21%|##        |…

Clean file runs/May26_12-03-48_3fa6313d89f7/events.out.tfevents.1685102630.3fa6313d89f7.28.4:  20%|#9        |…

Clean file training_args.bin:  27%|##7       | 1.00k/3.68k [00:00<?, ?B/s]

Clean file spiece.model:   0%|          | 1.00k/773k [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/231M [00:00<?, ?B/s]

In [18]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Currently logged in as: ethansimrm. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.15.3
wandb: Run data is saved locally in /kaggle/working/wandb/run-20230526_125119-y3g8mdug
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run graceful-vortex-6
wandb: ⭐️ View project at https://wandb.ai/ethansimrm/huggingface
wandb: 🚀 View run at https://wandb.ai/ethansimrm/huggingface/runs/y3g8mdug
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,0.808300,0.699417,19.696500,14.068900
2,0.794100,0.669866,20.098100,14.067400


TrainOutput(global_step=109258, training_loss=0.854210186620881, metrics={'train_runtime': 24377.052, 'train_samples_per_second': 71.711, 'train_steps_per_second': 4.482, 'total_flos': 3.155466927223603e+16, 'train_loss': 0.854210186620881, 'epoch': 2.0})

In [19]:
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/231M [00:00<?, ?B/s]

Upload file runs/May26_12-49-55_c5c674aa2a4e/events.out.tfevents.1685105479.c5c674aa2a4e.23.0:   0%|          …

To https://huggingface.co/ethansimrm/t5_small_prelim_emea_enfr
   749de76..78733a0  main -> main

To https://huggingface.co/ethansimrm/t5_small_prelim_emea_enfr
   78733a0..e602c03  main -> main



'https://huggingface.co/ethansimrm/t5_small_prelim_emea_enfr/commit/78733a0e7f7a309f0fed9b613673900745cd254a'